# Iceberg Classification Step 4: Model Serving

Notebook for serving The trained model for icerberg classification.

This notebook will send inference requests to a model serving instance that was exported by notebook number three ([Notebook number three](./Step3_Distributed_Training.ipynb)). This assumes that you have created a model serving instance of the model by using the hopsworks UI. You can find documentation on how to do this [here](https://hops.readthedocs.io/en/0.9/hopsml/model_serving.html)

This notebook is tested with the following ``configuration`` from hopsworks.
<div>
<img src="fig/step4_5_jupyter_config.png" width="900" align="center"/>
</div>

## Imports

In [1]:
import numpy as np
import tensorflow as tf

import hops
from hops import serving
from hops import kafka
import hsfs

# SparkSession available as 'spark'
print(
    f"-----------------------------------------------\n" \
    f"This notebook is tested with:\n" \
    f"  - TensorFlow {tf.__version__}.\n" \
    f"  - Hopsworks {hops.__version__}.\n" \
    f"  - Spark {spark.version}.\n"
)

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log
259,application_1619040920875_0292,pyspark,idle,Link,Link


SparkSession available as 'spark'.
-----------------------------------------------
This notebook is tested with:
  - TensorFlow 2.4.1.
  - Hopsworks 2.1.0.1.
  - Spark 2.4.3.2.

## Query Model Repository for best Ship Iceberg Classifer

In [2]:
from hops import model
from hops.model import Metric

MODEL_NAME = 'ship_iceberg_classifier'
EVALUATION_METRIC = 'val_accuracy'

best_model = model.get_best_model(MODEL_NAME, EVALUATION_METRIC, Metric.MAX)

print('Model name: ' + best_model['name'])
print('Model version: ' + str(best_model['version']))
print(best_model['metrics'])

Model name: ship_iceberg_classifier
Model version: 11
{'val_loss': '0.26797446608543396', 'train_loss': '0.25700217485427856', 'train_accuracy': '0.8842438459396362', 'val_accuracy': '0.8973606824874878'}

## Create Model Serving of Exported Model

In [3]:
# Create serving
model_path = '/Models/' + best_model['name']

# Since the name of the serving model cannot contain special characters.
serving_model_name = best_model['name'].title().replace('_', '')
print('Name of the serving model is {}'.format(serving_model_name))

response = serving.create_or_update(
                                        model_path,
                                        serving_model_name,
                                        serving_type="TENSORFLOW",
                                        model_version=best_model['version']
                                    )

Name of the serving model is ShipIcebergClassifier
Creating a serving for model ShipIcebergClassifier ...
Serving for model ShipIcebergClassifier successfully created

In [4]:
# List all available servings in the project
for s in serving.get_all():
    print(s.name)

ShipIcebergClassifier

In [5]:
# Get serving status
serving.get_status(serving_model_name)

'Running'

## Start Model Serving Server

In [6]:
if serving.get_status(serving_model_name) == 'Stopped':
    serving.start(serving_model_name)

In [7]:
import time

while serving.get_status(serving_model_name) != "Running":
    time.sleep(5) # Let the serving startup correctly
time.sleep(5)

In [8]:
# Get serving status
serving.get_status(serving_model_name)

'Running'

# Send Prediction Requests to the Served Model using Hopsworks REST API

## Make inference with randomly generated data

In [9]:
for i in range(20):
    data = {
                "signature_name": "serving_default",
                "instances": [np.random.rand(75, 75, 3).tolist()]
            }
    
    response = serving.make_inference_request(serving_model_name, data)
    print(response)

{'predictions': [[0.226102531]]}
{'predictions': [[0.208495587]]}
{'predictions': [[0.225897491]]}
{'predictions': [[0.184417903]]}
{'predictions': [[0.203099102]]}
{'predictions': [[0.197350144]]}
{'predictions': [[0.218518704]]}
{'predictions': [[0.223318249]]}
{'predictions': [[0.224881798]]}
{'predictions': [[0.207158417]]}
{'predictions': [[0.205661744]]}
{'predictions': [[0.224630475]]}
{'predictions': [[0.222884297]]}
{'predictions': [[0.231528491]]}
{'predictions': [[0.217980802]]}
{'predictions': [[0.208974451]]}
{'predictions': [[0.20945847]]}
{'predictions': [[0.213686436]]}
{'predictions': [[0.207000226]]}
{'predictions': [[0.23481369]]}

## Make inference with data from feature store

In [10]:
def create_severing_datasets():
    # Establish a connection with the Hopsworks feature store
    #     engine='training' is needed so that the executors in Spark can connect to feature store
    connection = hsfs.connection(engine='training')
    # Get the feature store handle for the project's feature store
    fs = connection.get_feature_store()
    

    TEST_FS_NAME = 'test_tfrecords_iceberg_classification_dataset'
    
    def decode(sample):
        """Decode each training sample.
        
        This funtionc decode each sample and return it in a format that is ready for training.
        
        Parameters:
        - sample: raw features of a data sample stored in a dictionary-like object
        
        Returns:
        - x: 'band_1', 'band_2', and 'band_avg' will be reshaped and stacked
             and form the input of the model
        - y: 'is_iceberg' will be the output of the model.
        """
        
        name_list = ['band_1', 'band_2', 'band_avg', 'is_iceberg']
        x = tf.stack([sample[name_list[0]], sample[name_list[1]], sample[name_list[2]]], axis=1)
        x = tf.reshape(x, [75, 75, 3])
        y = [tf.cast(sample[name_list[3]], tf.float32)]
        return x,y

    
    # Evaluation dataset in TFRecord format
    eval_ds = fs.get_training_dataset(name=TEST_FS_NAME).tf_data(target_name='is_iceberg')
    eval_ds = eval_ds.tf_record_dataset(process=False, batch_size=1, num_epochs=1)
    eval_ds_processed = eval_ds.map(decode).repeat(1).cache().batch(1).prefetch(tf.data.experimental.AUTOTUNE)
    
    return eval_ds_processed

In [11]:
def perform_inference(eval_ds, times=100):
    # Create iterator over the dataset
    dataset_iter = iter(eval_ds)
    
    for i in range(times):
        x, y = dataset_iter.get_next()
        request_data={}
        request_data['instances'] = [x.numpy().reshape((75, 75, 3)).tolist()]
        
        response = serving.make_inference_request("ShipIcebergClassifier", data=request_data, verb= ":predict")
        
        print("prediction: {:.10f},\t is_iceberg (ground truth): {}".format(response['predictions'][0][0], y[0][0]))       
        

In [12]:
eval_ds = create_severing_datasets()

Connected. Call `.close()` to terminate connection gracefully.

In [13]:
type(eval_ds)

<class 'tensorflow.python.data.ops.dataset_ops.PrefetchDataset'>

In [14]:
perform_inference(eval_ds)

prediction: 0.7165821190,	 is_iceberg (ground truth): 1.0
prediction: 0.7234406470,	 is_iceberg (ground truth): 1.0
prediction: 0.5848169330,	 is_iceberg (ground truth): 0.0
prediction: 0.0000000000,	 is_iceberg (ground truth): 0.0
prediction: 0.5488666300,	 is_iceberg (ground truth): 0.0
prediction: 0.0050038695,	 is_iceberg (ground truth): 0.0
prediction: 0.0084540844,	 is_iceberg (ground truth): 0.0
prediction: 0.0064917803,	 is_iceberg (ground truth): 0.0
prediction: 0.6524358990,	 is_iceberg (ground truth): 0.0
prediction: 0.0000000028,	 is_iceberg (ground truth): 0.0
prediction: 0.6434885260,	 is_iceberg (ground truth): 0.0
prediction: 0.9008358120,	 is_iceberg (ground truth): 1.0
prediction: 0.5515529510,	 is_iceberg (ground truth): 1.0
prediction: 0.4113817810,	 is_iceberg (ground truth): 0.0
prediction: 0.5667071340,	 is_iceberg (ground truth): 1.0
prediction: 0.8533031940,	 is_iceberg (ground truth): 1.0
prediction: 0.8942123650,	 is_iceberg (ground truth): 1.0
prediction: 0.

# End of Step 4.